<a href="https://colab.research.google.com/github/Achyuta-Harshavardhan/Keras_Tunner_ANN/blob/main/Keras_Tunner_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Keras Tunner for ANN**

In [ ]:
# Install Keras tunner
!pip install -U keras-tuner

In [27]:
# Import necessary packages
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense
from kerastuner.tuners import RandomSearch
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [28]:
df = pd.read_csv('/content/drive/MyDrive/Deep Learning/Dataset/Real_Combine.csv')
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [29]:
# Independet Fetures
x = df.iloc[:,:-1]

# Dependent feature
y = df.iloc[:,-1]

**Hyperparameters:**
1. How many number of hidden layers we should have?
2. How many number of neurons we should have in hidden layers?
3. Learning Rate

In [30]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [31]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    directory='project',
    project_name='Air Quality Index')

INFO:tensorflow:Reloading Oracle from existing project project/Air Quality Index/oracle.json


In [32]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [ ]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

In [34]:
model=tuner.get_best_models(num_models=1)[0]

In [ ]:
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))